In [1]:
import numpy as np
from matplotlib import pyplot as plt
from hdf5storage import loadmat,savemat
import pandas as pd

load networkx measurement outputs

In [2]:
outdict=loadmat('cc3_syn.mat')
cc3_syn=outdict['cc3_syn'] # 3nx x 2 syn x 4 condi x36 tr x 7 freq x 32 chan
# 3nx means 3 different networkx measurements; 2syn means 2 task types (synch and synco); 4 condi means 4 conditions; 

permutate 36condi+36independent

In [3]:
nx3gain10   =np.zeros((3,2,3,7,32)) # 3nx x 2syn x 3contrast condi x7freq x 32chan
nx3gain01   =np.zeros((3,2,3,7,32))
nx3counter10=np.zeros((3,2,3,7,32))
nx3counter01=np.zeros((3,2,3,7,32))

Code of the computation that produced current result of figure 6 in overleaf manuscript as of 5/2/2025


In [ ]:
nx3gain10   =np.zeros((3,2,3,7,32)) # 3nx x 2syn x 3contrast condi x7freq x 32chan
nx3gain01   =np.zeros((3,2,3,7,32))
nx3counter10=np.zeros((3,2,3,7,32))
nx3counter01=np.zeros((3,2,3,7,32))

rng = np.random.default_rng(seed =300)
for nx in range(3):
    for syn in range(2):
        print(syn)
        for contrast_condi in range(1,4):
            for freq in range(7):
                nx3gain10[nx,syn,contrast_condi-1,freq,:]=\
                    np.mean(cc3_syn[nx,syn,contrast_condi,:,freq,:],axis=0)/np.mean(cc3_syn[nx,syn,0,:,freq,:],axis=0)
                nx3gain01[nx,syn,contrast_condi-1,freq,:]=\
                    np.mean(cc3_syn[nx,syn,0,:,freq,:],axis=0)/np.mean(cc3_syn[nx,syn,contrast_condi,:,freq,:],axis=0) 
                sim10=np.zeros((100000,32))
                sim01=np.zeros((100000,32))
                combined_mat=np.concatenate((cc3_syn[nx,syn,0,:,freq,:], cc3_syn[nx,syn,contrast_condi,:,freq,:]),axis=0)
                for j in range(100000):
                    shuffled_combined = np.random.permutation(combined_mat)
                    sim10[j,:]=np.mean(shuffled_combined[36:72])/np.mean(shuffled_combined[0:36]) # all same values,but kept this as final results
                    sim01[j,:]=np.mean(shuffled_combined[0:36])/np.mean(shuffled_combined[36:72]) # this is equal to combine all channels and compute one ratio for each permuation instead for each channel separately
                    nx3counter10[nx,syn,contrast_condi-1,freq,:]=nx3counter10[nx,syn,contrast_condi-1,freq,:]+\
                        (nx3gain10[nx,syn,contrast_condi-1,freq,:] > sim10[j,:]) # check which channels are higher than permuted all channel average
                    nx3counter01[nx,syn,contrast_condi-1,freq,:]=nx3counter01[nx,syn,contrast_condi-1,freq,:]+\
                        (nx3gain01[nx,syn,contrast_condi-1,freq,:] > sim01[j,:])

In [ ]:
nx3permu2=dict()
nx3permu2['nx3gain10']  =nx3gain10
nx3permu2['nx3gain01']  =nx3gain01
nx3permu2['nx3counter10']=nx3counter10
nx3permu2['nx3counter01']=nx3counter01
savemat('nx3permu.mat',nx3permu2)

Corrected on 4/30/2025 but much less significant channels (decided to move on with results above and included it in the overleaf manuscript as fig 6)

In [ ]:
nx3gain10   =np.zeros((3,2,3,7,32)) 
nx3gain01   =np.zeros((3,2,3,7,32))
nx3counter10=np.zeros((3,2,3,7,32))
nx3counter01=np.zeros((3,2,3,7,32))

rng = np.random.default_rng(seed =300)
for nx in range(3):
    for syn in range(2):
        print(syn)
        for contrast_condi in range(1,4):
            for freq in range(7):
                nx3gain10[nx,syn,contrast_condi-1,freq,:]=\
                    np.mean(cc3_syn[nx,syn,contrast_condi,:,freq,:],axis=0)/np.mean(cc3_syn[nx,syn,0,:,freq,:],axis=0)
                nx3gain01[nx,syn,contrast_condi-1,freq,:]=\
                    np.mean(cc3_syn[nx,syn,0,:,freq,:],axis=0)/np.mean(cc3_syn[nx,syn,contrast_condi,:,freq,:],axis=0) 
                sim10=np.zeros((100000,32))
                sim01=np.zeros((100000,32))
                combined_mat=np.concatenate((cc3_syn[nx,syn,0,:,freq,:], cc3_syn[nx,syn,contrast_condi,:,freq,:]),axis=0)
                for j in range(100000):
                    shuffled_combined = np.random.permutation(combined_mat)
                    sim10[j,:]=np.mean(shuffled_combined[36:72],axis=0)/np.mean(shuffled_combined[0:36],axis=0) # corrected on 4/30/2025
                    sim01[j,:]=np.mean(shuffled_combined[0:36],axis=0)/np.mean(shuffled_combined[36:72],axis=0) # to do each channel separately
                    nx3counter10[nx,syn,contrast_condi-1,freq,:]=nx3counter10[nx,syn,contrast_condi-1,freq,:]+\
                        (nx3gain10[nx,syn,contrast_condi-1,freq,:] > sim10[j,:])
                    nx3counter01[nx,syn,contrast_condi-1,freq,:]=nx3counter01[nx,syn,contrast_condi-1,freq,:]+\
                        (nx3gain01[nx,syn,contrast_condi-1,freq,:] > sim01[j,:])
# 9 minutes

0
1
0
1
0
1


In [80]:
nx3permu2=dict()
nx3permu2['nx3gain10']  =nx3gain10
nx3permu2['nx3gain01']  =nx3gain01
nx3permu2['nx3counter10']=nx3counter10
nx3permu2['nx3counter01']=nx3counter01
savemat('nx3permu2.mat',nx3permu2)